In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/mohit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/mohit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
merdata = pd.read_csv('merdata1.csv', converters={'tokens': eval})
test = pd.read_csv('testdata1.csv', converters={'tokens': eval})

In [4]:
merdata

,Unnamed: 0,Unnamed: 0.1,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,general_cat,subcat_1,subcat_2,desc_len,tokens,logPrice,negative,neutral,positive,compound
0,0,0,1436222,Tarte rainforest after dark colored clay,1,Beauty/Makeup/Makeup Palettes,Tarte,36.0,1,"A brand-new, unused, unopened, undamaged item....",Beauty,Makeup,Makeup Palettes,83,"[brand, new, unused, unopened, undamaged, item...",3.610918,0.036,0.928,0.036,0.0000
1,1,1,402989,Mac mineralize skin finish,3,Beauty/Makeup/Face,MAC,15.0,1,"Color is gold deposit, about 85% of product left",Beauty,Makeup,Face,9,"[color, gold, deposit, product, left]",2.772589,0.000,1.000,0.000,0.0000
2,2,2,638275,White Case iPhone 6/6s,1,"Electronics/Cell Phones & Accessories/Cases, C...",Not known,3.0,1,New Ultra thin Candy TPU Silicone Rubber Soft,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins",8,"[new, ultra, thin, candy, tpu, silicone, rubbe...",1.386294,0.000,1.000,0.000,0.0000
3,3,3,1113629,Victoria's Secret push-up plunge,3,Women/Underwear/Bras,Victoria's Secret,18.0,1,VS sexy little thing multi way bras both size ...,Women,Underwear,Bras,36,"[sexy, little, thing, multi, way, bras, size, ...",2.944439,0.000,0.669,0.331,0.8126
4,4,4,328823,Disney Princess Toddler Boots Size 10,2,Kids/Girls 2T-5T/Shoes,Disney,13.0,1,New with out box!,Kids,Girls 2T-5T,Shoes,4,"[new, box]",2.639057,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037769,1037769,1037769,674578,Nike Dri fit hooded top,2,Women/Athletic Apparel/Shirts & Tops,Nike,20.0,1,Like new. Sz L. Thumb holes,Women,Athletic Apparel,Shirts & Tops,6,"[like, new, thumb, holes]",3.044522,0.000,0.615,0.385,0.3612
1037770,1037770,1037770,276890,Talk Red Hunter Boots,3,Women/Shoes/Boots,Hunter Boots,66.0,0,❤️❤️Sadly reselling because they are a little ...,Women,Shoes,Boots,20,"[❤️❤️sadly, reselling, little, small, boots, t...",4.204693,0.203,0.797,0.000,-0.4215
1037771,1037771,1037771,1069670,NWT FREE SHIP CC 2Beanie Hat Beige+Taup,1,Men/Men's Accessories/Hats,Not known,20.0,1,NWT Unisex Ribbed cable knit beanie slouchy th...,Men,Men's Accessories,Hats,57,"[nwt, unisex, ribbed, cable, knit, beanie, slo...",3.044522,0.099,0.807,0.094,-0.0516
1037772,1037772,1037772,35382,Winston cup snapback,3,Men/Men's Accessories/Hats,Not known,8.0,0,In great condition,Men,Men's Accessories,Hats,3,"[great, condition]",2.197225,0.000,0.196,0.804,0.6249


In [1]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [4]:
#operating on name column
from sklearn.feature_extraction.text import CountVectorizer
vectorizer1 = CountVectorizer(min_df = 10)
vectorizer1.fit(merdata['name'].values)
merdata_name = vectorizer1.transform(merdata['name'].values)
test_name = vectorizer1.transform(test['name'].values)

In [5]:
#normalizing item description length
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

normalizer.fit(merdata['desc_len'].values.reshape(-1,1))

x_train_words_des_norm = normalizer.transform(merdata['desc_len'].values.reshape(-1,1))
x_test_words_des_norm = normalizer.transform(test['desc_len'].values.reshape(-1,1))

In [13]:
x_train_words_des_norm[0]

array([1.])

In [6]:
from scipy.sparse import csr_matrix
x_dummies = csr_matrix(pd.get_dummies(merdata[['item_condition_id','shipping']],sparse=True).values)
x_test_dummies = csr_matrix(pd.get_dummies(test[['item_condition_id','shipping']],sparse=True).values)


In [7]:
vectorizer2 = CountVectorizer(min_df = 10)
vectorizer2.fit(merdata['brand_name'].values)
merdata_brandname = vectorizer2.transform(merdata['brand_name'].values)
test_brandname = vectorizer2.transform(test['brand_name'].values)

In [8]:
#General category Encoding
vectorizer3 = CountVectorizer(min_df = 10)
vectorizer3.fit(merdata['general_cat'].values)
merdata_gencat = vectorizer3.transform(merdata['general_cat'].values)
test_gencat = vectorizer3.transform(test['general_cat'].values)

In [9]:
merdata.isnull().any()

Unnamed: 0           False
train_id             False
name                 False
item_condition_id    False
category_name        False
brand_name           False
price                False
shipping             False
item_description     False
general_cat          False
subcat_1             False
subcat_2             False
desc_len             False
tokens               False
logPrice             False
dtype: bool

In [9]:
#Sub category1 Encoding
vectorizer4 = CountVectorizer(min_df = 10)
vectorizer4.fit(merdata['subcat_1'].values)
merdata_subcat1 = vectorizer4.transform(merdata['subcat_1'].values)
test_subcat1 = vectorizer4.transform(test['subcat_1'].values)

In [10]:
#Sub category2 Encoding
vectorizer5 = CountVectorizer(min_df = 10)
vectorizer5.fit(merdata['subcat_2'].values)
merdata_subcat2 = vectorizer5.transform(merdata['subcat_2'].values)
test_subcat2 = vectorizer5.transform(test['subcat_2'].values)

In [11]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [12]:
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 
            'won', "won't", 'wouldn', "wouldn't"]

In [13]:
from tqdm import tqdm
preprocessed_train_des = []
# tqdm is for printing the status bar
for sentance in tqdm(merdata['item_description'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
    preprocessed_train_des.append(sent.lower().strip())

# after preprocesing
preprocessed_train_des[20000]

100%|██████████| 1037774/1037774 [01:24<00:00, 12268.82it/s]


'like green excellent used condition no flaws piling stains'

In [14]:
preprocessed_train_des

NameError: name 'preprocessed_train_des' is not defined

In [14]:
from tqdm import tqdm
preprocessed_test_des = []
# tqdm is for printing the status bar
for sentance in tqdm(test['item_description'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
    preprocessed_test_des.append(sent.lower().strip())

# after preprocesing
preprocessed_test_des[20000]

100%|██████████| 444761/444761 [00:36<00:00, 12050.31it/s]


'royal purple black cute lingerie native 34b fair condition pink vs garter medium nwt polka push vs 32d'

In [15]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import nltk
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

train_sentiment = []; 
for sentence in tqdm(preprocessed_train_des):
    for_sentiment = sentence
    ss = sid.polarity_scores(for_sentiment)
    train_sentiment.append(ss)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mohit/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
100%|██████████| 1037774/1037774 [03:45<00:00, 4601.54it/s]


In [16]:
negative=[]
neutral=[]
positive=[]
compounding=[]
for i in train_sentiment:
    
    for polarity,score in i.items():
        if(polarity=='neg'):
            negative.append(score)
        if(polarity=='neu'):
            neutral.append(score)
        if(polarity=='pos'):
            positive.append(score)
        if(polarity=='compound'):
            compounding.append(score)

In [17]:
merdata['negative']=negative
merdata['neutral']=neutral
merdata['positive']=positive
merdata['compound']=compounding

In [18]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import nltk
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

X_test_sentiment = []; 
for sentence in tqdm(preprocessed_test_des):
    for_sentiment = sentence
    ss = sid.polarity_scores(for_sentiment)
    X_test_sentiment.append(ss)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mohit/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
100%|██████████| 444761/444761 [01:36<00:00, 4596.68it/s]


In [19]:
negative=[]
neutral=[]
positive=[]
compounding=[]
for i in X_test_sentiment:
    
    for polarity,score in i.items():
        if(polarity=='neg'):
            negative.append(score)
        if(polarity=='neu'):
            neutral.append(score)
        if(polarity=='pos'):
            positive.append(score)
        if(polarity=='compound'):
            compounding.append(score)

In [20]:
test['negative']=negative
test['neutral']=neutral
test['positive']=positive
test['compound']=compounding


In [23]:
test.to_csv('testdata1.csv')

In [16]:
y_train = merdata['price']

In [17]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

normalizer.fit(merdata['negative'].values.reshape(-1,1))

#X_train_price_norm = normalizer.transform(X_train['price'].values.reshape(-1,1))
X_train_neg_norm = normalizer.transform(merdata['negative'].values.reshape(-1,1))

X_test_neg_norm = normalizer.transform(test['negative'].values.reshape(-1,1))

print("After normalizations")
print(X_train_neg_norm.shape, y_train.shape)



After normalizations
(1037774, 1) (1037774,)


In [18]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

normalizer.fit(merdata['neutral'].values.reshape(-1,1))

#X_train_price_norm = normalizer.transform(X_train['price'].values.reshape(-1,1))
X_train_neu_norm = normalizer.transform(merdata['neutral'].values.reshape(-1,1))

X_test_neu_norm = normalizer.transform(test['neutral'].values.reshape(-1,1))

print("After normalizations")
print(X_train_neu_norm.shape, y_train.shape)

After normalizations
(1037774, 1) (1037774,)


In [19]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

normalizer.fit(merdata['positive'].values.reshape(-1,1))

#X_train_price_norm = normalizer.transform(X_train['price'].values.reshape(-1,1))
X_train_pos_norm = normalizer.transform(merdata['positive'].values.reshape(-1,1))

X_test_pos_norm = normalizer.transform(test['positive'].values.reshape(-1,1))

print("After normalizations")
print(X_train_pos_norm.shape, y_train.shape)

After normalizations
(1037774, 1) (1037774,)


In [20]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

normalizer.fit(merdata['compound'].values.reshape(-1,1))

#X_train_price_norm = normalizer.transform(X_train['price'].values.reshape(-1,1))
X_train_com_norm = normalizer.transform(merdata['compound'].values.reshape(-1,1))

X_test_com_norm = normalizer.transform(test['compound'].values.reshape(-1,1))

print("After normalizations")
print(X_train_com_norm.shape, y_train.shape)

After normalizations
(1037774, 1) (1037774,)


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=10,max_features=5000)
vectorizer.fit(preprocessed_train_des)
X_train_itemdes = vectorizer.transform(preprocessed_train_des)
X_test_itemdes = vectorizer.transform(preprocessed_test_des)

In [29]:
print("Shape of train matrix after one hot encodig ",X_train_itemdes.shape)
print("Shape of train matrix after one hot encodig ",X_test_itemdes.shape)

Shape of train matrix after one hot encodig  (1037774, 5000)
Shape of train matrix after one hot encodig  (444761, 5000)


In [17]:
from scipy.sparse import hstack

In [34]:
from scipy.sparse import hstack
X_train = hstack((merdata_name , merdata_brandname,merdata_gencat,merdata_subcat1,merdata_subcat2,X_train_itemdes,x_dummies,x_train_words_des_norm,X_train_neg_norm,X_train_neu_norm,X_train_pos_norm,X_train_com_norm)).tocsr()

X_test = hstack((test_name , test_brandname,test_gencat,test_subcat1,test_subcat2,X_test_itemdes,x_test_dummies,x_test_words_des_norm,X_test_neg_norm,X_test_neu_norm,X_test_pos_norm,X_test_com_norm)).tocsr()
y_train = merdata.price
print("Final data matrix")
print(X_train.shape,y_train.shape)
print(X_test.shape)

print("="*100)

Final data matrix
(1037774, 22769) (1037774,)
(444761, 22769)


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [27]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [25]:
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [41]:
ridge_reg = Ridge(solver='lsqr', fit_intercept=False, alpha=0.02,max_iter=200, normalize=False, tol=0.01)
ridge_reg.fit(X_train, y_train)

Ridge(alpha=0.02, fit_intercept=False, max_iter=200, solver='lsqr', tol=0.01)

In [42]:
y_test = ridge_reg.predict(X_test)
y_test

array([31.94287517, 52.77385551, 20.97622028, ..., 40.31837726,
       43.80470818, 22.79596895])

In [25]:
y_test = ridge_reg.predict(X_test)
y_test

array([41.43861857, 52.27681976, 18.87527044, ..., 14.59358146,
       37.06209213, 19.97182262])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
#from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
parameters = {"alpha":[0.01,0.1,0,1,10,100]}
ridgeReg = Ridge(solver = "lsqr", fit_intercept=False)
lr_reg = GridSearchCV(ridgeReg,param_grid =parameters,n_jobs=-1)
lr_reg.fit(X_train, y_train)

In [18]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
parameters = {"alpha": [0.001,0.01,0.1,0,1,10] , "l1_ratio":[0.3,0,4,0.5,0.6,0.7]}

model = SGDRegressor(loss = "squared_loss", learning_rate = "invscaling",max_iter = 200, penalty = "l2",fit_intercept = False)

lr_reg = GridSearchCV(model,param_grid = parameters , n_jobs = -1)
lr_reg.fit(X_train , y_train)

/home/mohit/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:1208: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


GridSearchCV(estimator=SGDRegressor(fit_intercept=False, max_iter=200),
             n_jobs=-1,
             param_grid={'alpha': [1e-08, 1e-05, 0.0001, 0.001, 0.01, 0.1, 0, 1,
                                   10, 100, 1000, 10000, 100000],
                         'l1_ratio': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]})

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [ ]:
param_dist = {'n_estimators': [10,50,100,150,160,200,300,350,400,500],
               'min_samples_split': [2,3,5,6,7,8],
               
              
              "max_depth":[None,10,20,40,60,80,100,120]
             
              }
regr1 = RandomForestRegressor()
n_iter_search = 100
regr1 = RandomizedSearchCV(regr1, param_distributions=param_dist,
                                     n_jobs=-1, n_iter=n_iter_search, cv=3)

regr1.fit(X_train, y_train)

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
gridParams = {
    'learning_rate': [ 0.1,0.2,0.3,0.4,0.5],
    'n_estimators': [100,150, 200,250,300,400,500],
    'num_leaves': [20,30,63,80,100,120],
    'boosting_type' : ['gbdt'],
    'max_depth' : [2,3,4,5,6,7,8]
}
lgbm_params ={'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_samples': 50, 'objective': 'regression'
        }
model = LGBMRegressor(**lgbm_params)
# Create the grid
grid = GridSearchCV(model, gridParams, verbose=1, cv=3, n_jobs=-1)
# Run the grid
grid.fit(X_train, y_train,
         
         early_stopping_rounds=100,
         verbose=True)

print('Best parameters found by grid search are:', grid.best_params_)
print('Best score found by grid search is:', grid.best_score_)




Fitting 3 folds for each of 1470 candidates, totalling 4410 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [30]:
from lightgbm import LGBMRegressor
lgbm_params ={'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_samples': 50, 'objective': 'regression','boosting_type': 'gbdt','learning_rate': 0.5,'max_depth': 8,'n_estimators': 500,'num_leaves': 80,
      }
model = LGBMRegressor(**lgbm_params)
model.fit(X_train, y_train, early_stopping_rounds=100,verbose=True)

ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

In [35]:
y_test = lr.predict(X_test)

In [36]:
y_test

array([42.25085769, 53.78152003, 18.84366425, ..., 23.28040138,
       36.99231316, 14.80814919])

In [43]:
test['price'] = y_test

In [44]:
Z = test[['id','price']]
Z

,id,price
0,1048243,31.942875
1,86986,52.773856
2,698316,20.976220
3,268868,32.691255
4,1360398,40.439743
...,...,...
444756,316678,21.183912
444757,795624,21.494045
444758,765638,40.318377
444759,766765,43.804708


In [45]:
Z.to_csv('ninth_submission.csv',index = False) 